# Parameterizing Notebooks & Building Workflows

**Objective:**
Learn how to make your Databricks notebooks dynamic by accepting parameters and how to build a simple workflow by calling one notebook from another.

**Scenario:**
We will create a Parent-Child notebook architecture.
1.  **Child Notebook:** Accepts a 'Department' parameter, reads employee data, filters it by that department, and writes it to a specific Delta table.
2.  **Parent Notebook:** Calls the Child Notebook multiple times with different department names, effectively running a data pipeline.

**Prerequisites:**
*   A running Databricks cluster.
*   The `emp.csv` file available in your Volume or DBFS (covered in previous videos).

## Part 1: Creating the Child Notebook Logic
*Note: In a real scenario, this code would be in a separate notebook file. For demonstration in this single file, we will simulate the logic blocks.*

Imagine the code below is inside a notebook named `Child_Notebook_Write_Data`.

In [ ]:
# --- LOGIC FOR CHILD NOTEBOOK STARTS HERE ---

# 1. Create a Text Widget to accept input
# Syntax: dbutils.widgets.text(name, defaultValue, label)
dbutils.widgets.text("dept_param", "Sales", "Department Name")

# 2. Get the value from the widget
current_dept = dbutils.widgets.get("dept_param")
print(f"Processing data for department: {current_dept}")

# 3. Read Data (Using the path from previous lessons)
# Update path if your file is elsewhere
file_path = "/Volumes/dev/bronze/managed_vol/files/emp.csv" 
df = spark.read.format("csv").option("header", "true").load(file_path)

# 4. Filter Data based on the parameter
# Using PySpark filter with upper() to handle case sensitivity
from pyspark.sql.functions import col, upper
df_filtered = df.filter(upper(col("department")) == upper(current_dept))

# 5. Write Data to a Delta Table specific to the department
# Table name will be dynamic: dev.bronze.dept_<dept_name>
table_name = f"dev.bronze.dept_{current_dept.lower()}"

# Check if we have data before writing
record_count = df_filtered.count()

if record_count > 0:
    df_filtered.write.format("delta").mode("overwrite").saveAsTable(table_name)
    print(f"Success: Wrote {record_count} records to table {table_name}")
    # 6. Exit with status
    dbutils.notebook.exit(f"Success: {record_count} records processed for {current_dept}")
else:
    print(f"No data found for department: {current_dept}")
    dbutils.notebook.exit(f"No Data: 0 records for {current_dept}")

# --- LOGIC FOR CHILD NOTEBOOK ENDS HERE ---

## Part 2: The Parent Notebook Logic (Orchestrator)

Now, we will use `dbutils.notebook.run` to call the child notebook. 

**Important:** Since we cannot actually call *this* current notebook recursively easily in a demo without side effects, imagine the code above is saved as `Child_Notebook`.

Below is the command you would use in a separate **Parent Notebook** to trigger the logic above.

In [ ]:
# Syntax: dbutils.notebook.run(path, timeout_seconds, arguments_dictionary)

# Example 1: Run for 'Sales'
# Note: This command will fail if 'Child_Notebook' does not actually exist in your workspace.
# You need to create the separate notebook file with the code from Cell 3 first.

# status_sales = dbutils.notebook.run("./Child_Notebook", 600, {"dept_param": "Sales"})
# print(status_sales)

# Example 2: Run for 'IT'
# status_it = dbutils.notebook.run("./Child_Notebook", 600, {"dept_param": "IT"})
# print(status_it)

## 3. Scheduling a Notebook Job
You can turn this parameterized notebook into a scheduled job directly from the UI.

1.  Click **Schedule** at the top right of the notebook.
2.  Click **Add Schedule**.
3.  Define the Job Name (e.g., "Daily Sales Data Load").
4.  Set the Schedule (e.g., Every Day at 8:00 AM).
5.  **Parameters:** You can add the key `dept_param` and value `Sales` here. This allows you to reuse the same code for different scheduled jobs (e.g., one job for Sales, one for Marketing).

## Summary
1.  **Widgets** allow notebooks to accept inputs at runtime (`dbutils.widgets`).
2.  **`dbutils.notebook.run`** allows you to chain notebooks, passing parameters and receiving exit values.
3.  **`dbutils.notebook.exit`** returns a value to the parent notebook and stops execution of the child notebook.
4.  This pattern is fundamental for building modular **ETL Pipelines** in Databricks.